# Métrica 1.1.05: "Congruencia entre el CP y la colonia/municipio"

In [2]:
import re
import urllib.parse
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure
import unicodedata
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- 1. CONFIGURACIÓN BÁSICA ---
METRIC_ID = "1_1_05_CP_VS_DOMICILIO"

# --- 2. LÓGICA PURA DE LA MÉTRICA ---

def normalize_string(s: str) -> str:
    """
    Quita acentos, caracteres especiales y convierte a mayúsculas para comparar.
    """
    if not isinstance(s, str): 
        return ""
    # Quita acentos
    s = ''.join(c for c in unicodedata.normalize('NFD', s)
                if unicodedata.category(c) != 'Mn')
    # Quita caracteres no alfanuméricos (deja letras, números y espacios)
    s_norm = re.sub(r'[^A-Z0-9\s]', '', s)
    return s_norm.upper().strip()

def validar_cp_en_mongo(db, cp: str, colonia_decl: str, municipio_decl: str) -> str:
    """
    Valida la congruencia del CP, Colonia y Municipio contra
    nuestra colección 'catalogo_cp' en MongoDB.
    """
    
    # 1. Validamos que tengamos todos los datos de entrada
    if not all([cp, colonia_decl, municipio_decl]):
        return "SIN_DATO"

    # 2. Normalizamos los datos declarados por el usuario
    cp_norm = str(cp).strip().zfill(5)
    colonia_norm = normalize_string(colonia_decl)
    municipio_norm = normalize_string(municipio_decl)

    catalogo_collection = db[SOURCE_COLLECTION_NAME]
    
    # 3. Buscamos el CP en nuestro catálogo (gracias al índice, esto es rápido)
    # Usamos 'd_codigo' como en tu muestra de datos
    registros_validos = list(catalogo_collection.find({"d_codigo": cp_norm}))

    if not registros_validos:
        # El CP declarado no existe en el catálogo oficial
        return "NO_CUMPLE_CP" 

    # 4. El CP existe. Ahora buscamos si la colonia y municipio coinciden
    encontrado = False
    for item in registros_validos:
        # Normalizamos los datos del catálogo oficial
        # Usamos 'd_asenta' y 'D_mnpio' como en tu muestra
        colonia_oficial = normalize_string(item.get("d_asenta"))
        municipio_oficial = normalize_string(item.get("D_mnpio"))
        
        # Comparamos: Municipio y Colonia deben ser una coincidencia exacta
        if municipio_norm == municipio_oficial and colonia_norm == colonia_oficial:
            encontrado = True
            break # Encontramos una coincidencia, no necesitamos seguir buscando
            
    # 5. Devolvemos el resultado final
    return "CUMPLE" if encontrado else "NO_CUMPLE"

# --- 3. EL FLUJO DEL WORKER (PROCESAMIENTO POR LOTES) ---
def procesar_todas_las_declaraciones_cp_declarante():
    """
    Procesa TODAS las declaraciones para la Métrica 1.1.05 (Domicilio Declarante)
    """
    client = None
    # Contadores
    total_documentos = 0
    cumple = 0
    no_cumple_cp = 0  # CP no existe
    no_cumple = 0     # CP existe, pero Col/Mun no coinciden
    sin_dato = 0
    
    try:
        # --- A. CONECTAR A LA BASE DE DATOS ---
        print(f"\nConectando a MongoDB en {DB_NAME}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ping') 
        db = client[DB_NAME]
        source_collection = db[SOURCE_COLLECTION_NAME]
        target_collection = db[METRICS_COLLECTION_NAME]
        
        # Verificamos que el catálogo exista antes de empezar
        if SOURCE_COLLECTION_NAME not in db.list_collection_names():
            print(f"--- ERROR CRÍTICO ---")
            print(f"La colección '{SOURCE_COLLECTION_NAME}' no existe.")
            print(f"Por favor, ejecuta primero el script para poblar el catálogo de CP.")
            print("---------------------")
            return

        print("¡Conexión exitosa!")
        print(f"--- INICIANDO PROCESAMIENTO (Métrica 1.1.05 - CP vs Domicilio Declarante) ---")

        # --- B. BUSCAR TODOS LOS DOCUMENTOS ---
        # Proyección optimizada para traer solo el domicilio del DECLARANTE
        proyeccion = {
            "_id": 1,
            "declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico": 1
        }
        
        # Iteramos sobre TODOS los documentos de la colección fuente
        for doc in source_collection.find({}, proyeccion):
            total_documentos += 1
            original_id = doc["_id"]
            cp, colonia, municipio = None, None, None
            
            try:
                # Navegación segura hacia la RUTA DEL DECLARANTE
                domicilio = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("domicilioDeclarante", {}).get("domicilioMexico")
                if isinstance(domicilio, dict):
                    cp = domicilio.get("codigoPostal")
                    colonia = domicilio.get("coloniaLocalidad")
                    municipio_obj = domicilio.get("municipioAlcaldia")
                    if isinstance(municipio_obj, dict):
                        municipio = municipio_obj.get("valor")
            except AttributeError:
                pass # Los valores se quedan en None

            # --- C. EJECUTAR MÉTRICA ---
            # Pasamos la conexión 'db' a la función de validación
            resultado_metrica = validar_cp_en_mongo(db, cp, colonia, municipio)
            
            # Actualizamos contadores
            if resultado_metrica == "CUMPLE": cumple += 1
            elif resultado_metrica == "NO_CUMPLE": no_cumple += 1
            elif resultado_metrica == "NO_CUMPLE_CP": no_cumple_cp += 1
            else: sin_dato += 1

            # --- D. IMPRIMIR RESULTADO INDIVIDUAL ---
            print(f"\nProcesando Documento ID: {original_id}")
            print("--- RESULTADO DEL ANÁLISIS ---")
            print(f"  > [Declarante] CP: '{cp}', Colonia: '{colonia}', Municipio: '{municipio}'")
            print(f"  > Resultado de la Métrica: {resultado_metrica}")
            print("---------------------------------")

            # --- E. GUARDAR RESULTADO EN LA COLECCIÓN 'metricas' ---
            filtro = { "_id": original_id }
            actualizacion = { "$set": { METRIC_ID: resultado_metrica } }
            # Usamos upsert=True para crear el doc en 'metricas' si no existe
            target_collection.update_one(filtro, actualizacion, upsert=True)
            print(f"¡Resultado guardado/actualizado en 'metricas' para el ID: {original_id}!")

        print("\n--- PROCESAMIENTO POR LOTES FINALIZADO ---")
        print("Resumen (Métrica 1.1.05):")
        print(f"  > Documentos Totales Procesados: {total_documentos}")
        print(f"  > Métrica 'CUMPLE': {cumple}")
        print(f"  > Métrica 'NO_CUMPLE' (Col/Mun no coinciden): {no_cumple}")
        print(f"  > Métrica 'NO_CUMPLE_CP' (CP no existe): {no_cumple_cp}")
        print(f"  > Métrica 'SIN_DATO': {sin_dato}")

    except ConnectionFailure: print("Error: No se pudo conectar a la base de datos.")
    except OperationFailure as e: print(f"Error en la operación de la base de datos: {e.details}")
    except Exception as e: print(f"Ocurrió un error inesperado: {e}")
    finally:
        if client: client.close(); print("Conexión cerrada.")

# --- 4. EJECUTAR EL SCRIPT ---
if __name__ == "__main__":
    procesar_todas_las_declaraciones_cp_declarante()


Conectando a MongoDB en sistema1...
¡Conexión exitosa!
--- INICIANDO PROCESAMIENTO (Métrica 1.1.05 - CP vs Domicilio Declarante) ---

Procesando Documento ID: 68f81b8800535f910a29f694
--- RESULTADO DEL ANÁLISIS ---
  > [Declarante] CP: 'None', Colonia: 'None', Municipio: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f694!

Procesando Documento ID: 68f81b8800535f910a29f695
--- RESULTADO DEL ANÁLISIS ---
  > [Declarante] CP: 'None', Colonia: 'None', Municipio: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f695!

Procesando Documento ID: 68f81b8800535f910a29f696
--- RESULTADO DEL ANÁLISIS ---
  > [Declarante] CP: 'None', Colonia: 'None', Municipio: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas

KeyboardInterrupt: 